### Importing the File
1. import pandas
2. export from VAN as txt, and save to this directory
3. read table 

In [3]:
import pandas as pd

In [4]:
#### if not Fryda, make sure to change the working directory to where you are saving everything in your computer
### change the name to the folder where you save the files and also where you will launch jupyter notebook

# os.chdir('../')


In [5]:
###reading a txt file
#### had to add engine = "python" to fix a unicode error

df = pd.read_table("/Users/frydafly/Downloads/serverupdate/testCH-20609889456.txt", engine = "python")

### for reading a csv file, see below
# ch = pd.read_csv("")

In [13]:
#### take a look at all the columns
df.head(2)

,Voter File VANID,DWID,VoterName,ResultShortName,CreatedBy,CanvassedBy,CommitteeName,DateCreated,DateCanvassed,ContactTypeName,Campaign,Content,InputTypeName
0,23763,25730899.0,"Marquez, Sammy A",Canvassed,Hustle STV API,"API, Hustle STV",Minkwon,12/02/2020,12/02/2020,SMS Text,NaN,NaN,API
1,27122,27804888.0,"Dejesus, Monica Lynn",Canvassed,Hustle STV API,"API, Hustle STV",Minkwon,12/02/2020,12/02/2020,SMS Text,NaN,NaN,API


In [29]:
## count number of rows

df.count()

Voter File VANID    45446
DWID                44521
VoterName           45446
ResultShortName     45446
CreatedBy           45446
CanvassedBy         45440
CommitteeName       45446
DateCreated         45446
DateCanvassed       45446
ContactTypeName     45446
Campaign                0
Content                 0
InputTypeName       45446
dtype: int64

## Clean Dataframe
1. Drop extra columns
2. Reorder Columns 
3. Convert DWID from float to integer
4. uppercase Result, ContactType, InputType
5. Remove NaN, and make blank
6. Standardize committee names based on translation table?

In [12]:
# drop VoterName and Content Columns

df1 = df.drop(columns=['VoterName', 'Content'])

#look at the new columns
df1.head(2)

,Voter File VANID,DWID,ResultShortName,CreatedBy,CanvassedBy,CommitteeName,DateCreated,DateCanvassed,ContactTypeName,Campaign,InputTypeName
0,23763,25730899.0,Canvassed,Hustle STV API,"API, Hustle STV",Minkwon,12/02/2020,12/02/2020,SMS Text,NaN,API
1,27122,27804888.0,Canvassed,Hustle STV API,"API, Hustle STV",Minkwon,12/02/2020,12/02/2020,SMS Text,NaN,API


In [11]:
# reorder the columns

columnsTitles = ['Voter File VANID', 'DWID', 'ResultShortName', 'DateCreated', 'DateCanvassed', 'ContactTypeName', 'InputTypeName', 'CommitteeName', 'CreatedBy', 'CanvassedBy', 'Campaign']
df2 = df1.reindex(columns=columnsTitles)

#look at new columns
df2.head(2)

,Voter File VANID,DWID,ResultShortName,DateCreated,DateCanvassed,ContactTypeName,InputTypeName,CommitteeName,CreatedBy,CanvassedBy,Campaign
0,23763,25730899.0,Canvassed,12/02/2020,12/02/2020,SMS Text,NaN,Minkwon,Hustle STV API,"API, Hustle STV",NaN
1,27122,27804888.0,Canvassed,12/02/2020,12/02/2020,SMS Text,NaN,Minkwon,Hustle STV API,"API, Hustle STV",NaN


In [9]:
# Remove NaN

df2.fillna('', inplace=True)

In [31]:
# uppercase for Result, ContactType, InputType

df2['ResultShortName'] = df2['ResultShortName'].str.upper() 
df2['ContactTypeName'] = df2['ContactTypeName'].str.upper() 
df2['InputTypeName'] = df2['InputTypeName'].str.upper() 

#display results
df2.head(2)

,Voter File VANID,DWID,ResultShortName,DateCreated,DateCanvassed,ContactTypeName,InputTypeName,CommitteeName,CreatedBy,CanvassedBy,Campaign
0,23763,25730899,CANVASSED,12/02/2020,12/02/2020,SMS TEXT,NaN,MinKwon,Hustle STV API,"API, Hustle STV",NaN
1,27122,27804888,CANVASSED,12/02/2020,12/02/2020,SMS TEXT,NaN,MinKwon,Hustle STV API,"API, Hustle STV",NaN


In [32]:
# check types to make sure it makes sense

df2.dtypes

Voter File VANID     int64
DWID                 int64
ResultShortName     object
DateCreated         object
DateCanvassed       object
ContactTypeName     object
InputTypeName       object
CommitteeName       object
CreatedBy           object
CanvassedBy         object
Campaign            object
dtype: object

In [33]:
##convert DWID to int

#fill na by 0, since that caused an issue the first time
df2['DWID'] = df2['DWID'].fillna(0)

#convert DWID to int
df2['DWID'] = df2['DWID'].astype('int64') 

##convert inputtypename & campaign to object
df2 = df2.astype({"InputTypeName":'object', "Campaign":'object'})  

#display types again
df2.dtypes

Voter File VANID     int64
DWID                 int64
ResultShortName     object
DateCreated         object
DateCanvassed       object
ContactTypeName     object
InputTypeName       object
CommitteeName       object
CreatedBy           object
CanvassedBy         object
Campaign            object
dtype: object

## Clean Out Org Names
1. Remove orgs that are not NYCET Partner Orgs
2. Rename org names to fit names in server

In [34]:
# List distinct CommitteeNames

df2.CommitteeName.unique()

array(['MinKwon', 'WE ACT', 'MRNY', 'NWBCCC', 'GH', 'LICET'], dtype=object)

In [35]:
## Remove orgs that are not NYCET partner org

#list out index for rows that are not partners
indexNames = df2[df2['CommitteeName'] == 'Planned Parenthood c3'].index

#drop rows with these indexes
df2.drop(indexNames, inplace=True)

#check the distinct committeenames again
df2.CommitteeName.unique()

array(['MinKwon', 'WE ACT', 'MRNY', 'NWBCCC', 'GH', 'LICET'], dtype=object)

In [36]:
# rename existing names to fit orgnames in server

df2['CommitteeName'] = df['CommitteeName'].replace(['Minkwon','Make the Road'],['MinKwon', 'MRNY'])

#display new names
df2.CommitteeName.unique()

array(['MinKwon', 'WE ACT', 'MRNY', 'NWBCCC', 'GH', 'LICET'], dtype=object)

## Narrow to Proper Date Range

In [37]:
## this isn't ready yet, and we didn't need it yet, so I'll skip

## Check our work

In [38]:
# count number of rows

df2.count()

Voter File VANID    45440
DWID                45440
ResultShortName     45440
DateCreated         45440
DateCanvassed       45440
ContactTypeName     45440
InputTypeName           0
CommitteeName       45440
CreatedBy           45440
CanvassedBy         45440
Campaign                0
dtype: int64

## Print New CSV

In [40]:
df2.to_csv(r'/Users/frydafly/Downloads/serverupdate/newfile.csv', index = False)